In [ ]:
import enum
import math
import os
import random
from typing import List, Text, Dict
from google.colab import auth
from google.api_core import retry
from IPython.display import Image
from matplotlib import pyplot as plt
from numpy.lib import recfunctions as rfn
import ee
import google
import io
import numpy as np
import requests
import tensorflow as tf
import ee
from datetime import datetime, timedelta

import geemap
random.seed(123)

In [ ]:
# PROJECT = 'wildfire-lab'
# auth.authenticate_user()
# credentials, _ = google.auth.default()

In [ ]:
ee.Authenticate()
ee.Initialize(project='wildfire-lab')

In [ ]:
start_date = ee.Date("2016-07-18")
end_date = ee.Date("2016-08-07")
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(start_date.advance(-1, "day"), end_date)

In [ ]:
import pandas as pd
fires = pd.read_csv("request_data.csv", index_col=0, parse_dates=[1, 2])
fire = fires[["4416704" in fire for fire in fires.fire_id]].iloc[0]
fire.head()

fire_id       2016_4416704_PIONEER
start_date     2016-07-18 19:00:00
end_date       2016-08-07 19:00:00
poo_lat                      43.95
poo_lon                    -115.76
Name: 2444, dtype: object

## Constants

In [ ]:
DEFAULT_KERNEL_SIZE = 64

In [ ]:
class DataType(enum.Enum):
  ELEVATION_SRTM = 1
  VEGETATION_VIIRS = 2
  DROUGHT_GRIDMET = 3
  WEATHER_ERA5 = 4
  WEATHER_GRIDMET = 5
  FIRE_MODIS = 6
  POPULATION = 7
DATA_SOURCES = {
    DataType.ELEVATION_SRTM: 'USGS/SRTMGL1_003',
    DataType.VEGETATION_VIIRS: 'NOAA/VIIRS/001/VNP13A1',
    DataType.DROUGHT_GRIDMET: 'GRIDMET/DROUGHT',
    DataType.WEATHER_ERA5: 'ECMWF/ERA5/DAILY',
    DataType.WEATHER_GRIDMET: 'IDAHO_EPSCOR/GRIDMET',
    DataType.FIRE_MODIS: 'MODIS/006/MOD14A1',
    DataType.POPULATION: 'CIESIN/GPWv411/GPW_Population_Density'
}
COORDINATES = {
    # Used as input to ee.Geometry.Rectangle().
    'US': [-124, 24, -73, 49]
}
DATA_BANDS = {
    DataType.ELEVATION_SRTM: ['elevation'],
    DataType.VEGETATION_VIIRS: ['NDVI'],
    DataType.DROUGHT_GRIDMET: ['pdsi'],
    DataType.WEATHER_ERA5: [
        'mean_2m_air_temperature',
        'total_precipitation',
        'u_component_of_wind_10m',
        'v_component_of_wind_10m',
    ],
    DataType.WEATHER_GRIDMET: [
        'pr',
        'sph',
        'th',
        'tmmn',
        'tmmx',
        'vs',
        'erc',
    ],
    DataType.FIRE_MODIS: ['FireMask'],
    DataType.POPULATION: ['population_density']
}

In [ ]:
def get_image(data_type):
  """Gets an image corresponding to `data_type`.

  Args:
    data_type: A specifier for the type of data.

  Returns:
    The EE image correspoding to the selected `data_type`.
  """
  return ee.Image(DATA_SOURCES[data_type]).select(DATA_BANDS[data_type])


def get_image_collection(data_type):
  """Gets an image collection corresponding to `data_type`.

  Args:
    data_type: A specifier for the type of data.

  Returns:
    The EE image collection corresponding to `data_type`.
  """
  return ee.ImageCollection(DATA_SOURCES[data_type]).select(
      DATA_BANDS[data_type])

In [ ]:
N = 1
df = fires.iloc[:N]
start_date = df['start_date'].iloc[0]
start_date_filter = (start_date - timedelta(days=5)).strftime('%Y-%m-%d')
end_date = df['start_date'].iloc[-1]
end_date_filter = (end_date + timedelta(days=25)).strftime('%Y-%m-%d')

In [ ]:
SCALE = 1000
proj = ee.Projection('EPSG:4326').atScale(SCALE).getInfo()

In [ ]:
images = get_image_collection(DataType.WEATHER_GRIDMET)

In [ ]:
feature_collection = ee.FeatureCollection([])

In [ ]:
PATCH_SIZE = 64000

In [ ]:
lag = 5
window_start = ee.Date(fire['start_date'].strftime("%Y-%m-%d"))
window = 21
window_end = window_start.advance(window, "days")
center_point = fire[['poo_lon', 'poo_lat']].values
geom_point = ee.Geometry.Point(*center_point)
bbox = geom_point.buffer(PATCH_SIZE).bounds()
weather = images.filterDate(window_start.advance(-lag, "days"), window_end).filterBounds(bbox)

In [ ]:
weather.toArrayPerBand(axis=2)

/usr/local/lib/python3.10/dist-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'ImageCollection.toArrayPerBand: Axis 2 exceeds 0 dimensionality of band 'pr'.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [ ]:
RADIUS = 64000//2
geometry = geom_point.buffer(RADIUS).bounds()

## Kernel

In [ ]:
maximumTemperature = weather.select('tmmx');
maximumTemperatureVis = {
  "min": 290.0,
  "max": 314.0,
  "palette": ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c'],
};
m = geemap.Map()
m.setCenter(*center_point, 9);
m.addLayer(maximumTemperature, maximumTemperatureVis, 'Maximum Temperature', opacity=0.8)
m.addLayer(geometry, {'color': '000000'}, 'Bbox', opacity=0.2)
m.addLayer(bbox, {'color': '000000'}, 'Kernel scope', opacity=0.2)
m

Map(center=[43.95, -115.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [ ]:
def convert_features_to_arrays(
    image_list,
    kernel_size = 64,
):
  """Converts a list of EE images into `(kernel_size x kernel_size)` tiles.

  Args:
    image_list: The list of EE images.
    kernel_size: The size of the tiles (kernel_size x kernel_size).

  Returns:
    An EE image made of (kernel_size x kernel_size) tiles.
  """
  feature_stack = ee.Image.cat(image_list).float()
  kernel_list = ee.List.repeat(1, kernel_size)  # pytype: disable=attribute-error
  kernel_lists = ee.List.repeat(kernel_list, kernel_size)  # pytype: disable=attribute-error
  kernel = ee.Kernel.fixed(kernel_size, kernel_size, kernel_lists)
  return feature_stack.neighborhoodToArray(kernel)

In [ ]:
maximumTemperature = weather_image.select('20160713_tmmx');
maximumTemperatureVis = {
  "min": 290.0,
  "max": 314.0,
  "palette": ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c'],
};
m = geemap.Map()
m.setCenter(*center_point, 9);
m.addLayer(maximumTemperature, maximumTemperatureVis, 'Maximum Temperature', opacity=0.8)
m.addLayer(geometry, {'color': '000000'}, 'Bbox', opacity=0.2)
m.addLayer(bbox, {'color': '000000'}, 'Kernel scope', opacity=0.2)
m

Map(center=[43.95, -115.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [ ]:
kerneled = convert_features_to_arrays([weather_image])

In [ ]:
weather_image.select("20160713_pr")

In [ ]:
weather_image.select("20160713_pr").sampleRegions(ee.FeatureCollection([geometry]), scale=1000)

In [ ]:
fc = ee.FeatureCollection(ee.Feature(geometry))

In [ ]:
pixel_fc = kerneled.sampleRegions(collection=fc, scale=1000)
pixel_fc

In [ ]:

bucket = "wildfire-lab"
folder = "batch_export_test_25"
description = "sampleRegions"
file_format = "TFRecord"
task = ee.batch.Export.table.toCloudStorage(
      collection=pixel_fc,
      description=description,
      bucket=bucket,
      fileNamePrefix=os.path.join(folder, description),
      fileFormat=file_format)
task.start()

In [ ]:
sampleRect = ee.FeatureCollection(kerneled.sampleRectangle(geometry))
bucket = "wildfire-lab"
folder = "batch_export_test_25"
description = "sampleRectangle"
file_format = "TFRecord"
task = ee.batch.Export.table.toCloudStorage(
      collection=sampleRect,
      description=description,
      bucket=bucket,
      fileNamePrefix=os.path.join(folder, description),
      fileFormat=file_format)
task.start()


In [ ]:
geometry

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.bounds",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "Geometry.buffer",
          "arguments": {
            "distance": {
              "constantValue": 32000
            },
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Point",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      -115.76,
                      43.95
                    ]
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

In [ ]:
f = kerneled.sampleRectangle(geometry)
f.getInfo()

In [ ]:
test = kerneled.select("20160713_pr").sample(geometry)

In [ ]:
test

In [ ]:
fc = kerneled.sample(geometry, scale=1000, seed=0)

In [ ]:
bucket = "wildfire-lab"
folder = "batch_export_test_25"
description = "sample"
file_format = "TFRecord"
task = ee.batch.Export.table.toCloudStorage(
      collection=feature_collection,
      description=description,
      bucket=bucket,
      fileNamePrefix=os.path.join(folder, description),
      fileFormat=file_format)
task.start()


## TEST output

In [ ]:
def parse_tfrecord(example_proto):
  """Parse a serialized example."""
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)
OUTPUT_FILE = "gs://wildfire-lab/batch_export_test_25/sampleRectangle.tfrecord.gz"
dataset = tf.data.TFRecordDataset(OUTPUT_FILE, compression_type="GZIP")

In [ ]:
next(iter(dataset))

### Other method: clipBoundsAndScale

In [ ]:
image = maximumTemperature.first().clipToBoundsAndScale(geometry, 64, 64)
maximumTemperatureVis = {
  "min": 290.0,
  "max": 314.0,
  "palette": ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c'],
};
m = geemap.Map()
m.setCenter(*center_point, 9);
m.addLayer(image, maximumTemperatureVis, 'Maximum Temperature', opacity=0.8)
m.addLayer(geometry, {'color': '000000'}, 'Bbox', opacity=0.2)
m.addLayer(bbox, {'color': '000000'}, 'Kernel scope', opacity=0.2)
m

Map(center=[43.95, -115.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [ ]:
fire_weather = weather.map(lambda x: x.clipToBoundsAndScale(geometry, 64, 64)).toBands()

In [ ]:
fire_weather

In [ ]:
def samp_arr_img(arr_img):
  point = ee.Geometry.Point([-151, 42])
  return arr_img.sample(point, 500).first().get('array')
samp_arr_img(fire_weather.toArray())

In [ ]:
bands = fire_weather.bandNames().getInfo()

In [ ]:
bucket = "wildfire-lab"
folder = "batch_export_test_28"
description = "fire_1"
file_format = "TFRecord"
image_export_options = {
  'patchDimensions': [64, 64],
  'maxFileSize': 104857600,
  'compressed': True
}
task = ee.batch.Export.image.toCloudStorage(
      image=fire_weather,
      description=description,
      bucket=bucket,
      scale=1000,
      fileNamePrefix=os.path.join(folder, description),
      fileFormat=file_format,
      formatOptions=image_export_options)
task.start()

In [ ]:
train_dataset = tf.data.TFRecordDataset("gs://"+bucket+"/"+os.path.join(folder, "fire_1.tfrecord.gz"), compression_type='GZIP')
# Print the first record to check.
print(iter(train_dataset).next())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
columns = [
  tf.io.FixedLenFeature(shape=[64,64], dtype=tf.float32) for k in bands
]

# Dictionary with names as keys, features as values.
features_dict = dict(zip(bands, columns))

print(features_dict)

{'20160713_pr': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160713_sph': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160713_th': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160713_tmmn': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160713_tmmx': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160713_vs': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160713_erc': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160714_pr': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160714_sph': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160714_th': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160714_tmmn': FixedLenFeature(shape=[64, 64], dtype=tf.float32, default_value=None), '20160714_tmmx': FixedLenFeature(shape=[64, 

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  return parsed_features

# Map the function over the dataset.
parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)



In [ ]:
next(iter(parsed_dataset))

{'20160713_erc': <tf.Tensor: shape=(64, 64), dtype=float32, numpy=
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [60., 60., 60., ..., 57., 59., 59.],
        [60., 60., 60., ..., 57., 59., 59.],
        ...,
        [62., 62., 62., ..., 62., 63., 63.],
        [62., 62., 62., ..., 62., 63., 63.],
        [62., 62., 62., ..., 62., 63., 63.]], dtype=float32)>,
 '20160713_pr': <tf.Tensor: shape=(64, 64), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 '20160713_sph': <tf.Tensor: shape=(64, 64), dtype=float32, numpy=
 array([[0.     , 0.     , 0.     , ..., 0.     , 0.     , 0.     ],
        [0.00579, 0.00579, 0.00579, ..., 0.00469, 0.00453, 0.00453],
        [0.00579, 0.00579, 0.00579, ..., 0.00469, 0.00453, 0.00453],
        ...,
        [0.005

In [ ]:
images = maximumTemperature.map(lambda x: x.clipToBoundsAndScale(geometry, 64, 64))
maximumTemperatureVis = {
  "min": 290.0,
  "max": 314.0,
  "palette": ['d8d8d8', '4addff', '5affa3', 'f2ff89', 'ff725c'],
};
m = geemap.Map()
m.setCenter(*center_point, 9);
m.addLayer(images.first(), maximumTemperatureVis, 'Maximum Temperature', opacity=0.8)
m.addLayer(geometry, {'color': '000000'}, 'Bbox', opacity=0.2)
m

Map(center=[43.95, -115.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…